In [1]:
import modin.pandas as pd
import re
import lxml.html
import re

In [2]:
df_raw = pd.read_parquet("../data/df_raw/")

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

(raylet) /opt/conda/envs/stackoverflow/lib/python3.8/site-packages/ray/dashboard/agent.py:152: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(raylet)   if LooseVersion(aiohttp.__version__) < LooseVersion("4.0.0"):
(raylet) /opt/conda/envs/stackoverflow/lib/python3.8/site-packages/ray/dashboard/agent.py:152: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(raylet)   if LooseVersion(aiohttp.__version__) < LooseVersion("4.0.0"):
(raylet) /opt/conda/envs/stackoverflow/lib/python3.8/site-packages/ray/dashboard/agent.py:152: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(raylet)   if LooseVersion(aiohttp.__version__) < LooseVersion("4.0.0"):
(raylet) /opt/conda/envs/stackoverflow/lib/python3.8/site-packages/ray/dashboard/agent.py:152: D

In [3]:
df_raw.columns

Index(['Id', 'AcceptedAnswerId', 'Title', 'Body', 'Tags', 'ViewCount',
       'AnswerCount', 'CommentCount', 'Score', 'CreationDate', 'AnswerBody'],
      dtype='object')

In [4]:
!ls

00_data_fetch_bq.ipynb	   01_b_setup_new.ipynb   02_indexing_faiss.ipynb  old
00_data_fetch_spark.ipynb  01_data_cleanup.ipynb  03_searching_es.ipynb
01_b_setup.ipynb	   02_indexing_es.ipynb   Untitled.ipynb


In [5]:
df_raw.head()

,Id,AcceptedAnswerId,Title,Body,Tags,ViewCount,AnswerCount,CommentCount,Score,CreationDate,AnswerBody
0,17123652,17124724,hierarchical encryption scheme,"<p>I am in need of the ""hierarchical"" encrypti...",<cryptography><key><hierarchical>,631,1,0,2,2013-06-15 12:29:50.987,<p>A partial solution. You own the master key...
1,44577139,44577209,Uncaught TypeError: this.source is not a function,<p>I want to prelaod all the customers and giv...,<ajax><jquery-ui>,3396,1,0,1,2017-06-15 21:14:46.990,<p>Initialize <strong>autocomplete</strong> af...
2,45162881,45162984,Class App\Http\Controllers\ does not exist,<p>This is my Route: </p>\n\n<pre><code> Route...,<php><laravel><controller><routes>,32241,3,1,2,2017-07-18 09:35:26.630,<p>At the first of controller you do not need ...
3,1887841,1890092,Grails startup is slow,<p>Help! I'm porting a large ruby app to Grail...,<grails>,9592,5,0,28,2009-12-11 12:43:03.790,"<p>Unfortunately, I am not sure too much can b..."
4,8151129,8151158,AlertDialog - trying to understand this syntax,<p>This is code from the book sample:</p>\n\n<...,<java><android>,490,5,1,2,2011-11-16 11:46:42.137,<pre><code>// Create a builder\nAlertDialog.Bu...


In [6]:
regex = r"""
	<pre>.*?</pre>
	"""

def clean_text(snippet:str):
    snippet = re.sub(pattern=regex, repl = '[CODE]', string = snippet,  flags = re.IGNORECASE | re.DOTALL | re.MULTILINE | re.VERBOSE )
    
    snippet = str(lxml.html.fromstring(snippet).text_content())
    
    return snippet

def parse_tags(content:str):
    return re.findall(r'<(.+?)>',content)

In [7]:
clean_text(f"""

<p>I was asked to create a singleton that will..</p>.

<pre><code>KDF </code></pre>

<p> test </p>

""")

'I was asked to create a singleton that will...\n\n[CODE]\n\n test \n\n'

In [8]:
df = df_raw

In [9]:
df['QuestionBody'] =  df['Body'].apply(clean_text)

In [10]:
df['AnswerBody'] =  df['AnswerBody'].apply(clean_text)

In [11]:
df['Tags'] = df['Tags'].apply(parse_tags)

In [12]:
df.to_parquet("../data/df_processed/")

In [13]:
!ls ../data/df_processed/

part-0000.snappy.parquet  part-0006.snappy.parquet  part-0012.snappy.parquet
part-0001.snappy.parquet  part-0007.snappy.parquet  part-0013.snappy.parquet
part-0002.snappy.parquet  part-0008.snappy.parquet  part-0014.snappy.parquet
part-0003.snappy.parquet  part-0009.snappy.parquet  part-0015.snappy.parquet
part-0004.snappy.parquet  part-0010.snappy.parquet
part-0005.snappy.parquet  part-0011.snappy.parquet
